#### Setup

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Content Management 

![Automation!](./imgs/automation.jpg)

- Users can host a wide variety of content
- Managing content is essential to a well run system
- There are multiple [item type](https://developers.arcgis.com/rest/users-groups-and-items/items-and-item-types.htm) that Enterprise/ArcGIS Online can host
- Items have a complex [relationship](https://developers.arcgis.com/rest/users-groups-and-items/relationship-types.htm) that is often not visible to the regular user

## Why Automate Conent Management?

- time saving
- increated productivity
- consistency and accuracy
- version control and tracking
- scalability


## Understanding your Site

In [ ]:
from arcgis.gis import GIS, ContentManager
from arcgis.gis import ItemTypeEnum, ItemProperties
from arcgis.gis import Item
import pandas as pd

In [ ]:
gis:GIS = GIS(profile='your_online_profile')
content_manager:ContentManager = gis.content

### Searching and Exploring
![Exploring!](./imgs/exploring.jpg)

- If you don't know what you have, how can you move forward?
- The `search` and `advanced_search` methods allow you to explore, understand and find content
- Built-in reports provide insights

#### Basic Search

- Gives a general insight into Items located on your site
- Need to specify the number of items returned
- Always searches within your organization first

In [ ]:
items = content_manager.search("Roads")
items

##### Filtering by Type

- When you want a specific type of data, use `item_type`

In [ ]:
#Getting Help in Jupyter Notebooks:
content_manager.search?

In [ ]:
items = content_manager.search(query="Roads",
                               item_type="Web Map", 
                               outside_org=True,
                               max_items=100)
items

##### Filtering by a User

In [ ]:
items = content_manager.search(query=f"Roads owner:{gis.users.me.username}",
                               item_type="Web Map", 
                               enrich=True)
items

#### Advanced Search

- fully customize the search experience

In [ ]:
items = content_manager.advanced_search(query="Roads type:'Web Map'", as_dict=True)
items['results'][0]

##### Gathering Statistics

- How do we know what is on our organization as a glance?
- Advanced statistics gives insight into what is on our site

In [ ]:
counts = content_manager.advanced_search(query=f"accountid:{gis.properties.id}", 
                                count_fields="type, access", 
                                count_size=200)

In [ ]:
counts

###### Exploring the Access Types of Items

In [ ]:
data = counts['counts'][0]['fieldValues']
df = pd.DataFrame(data, )
df.columns=['access', 'count']
df.plot.bar(x='access', y='count', rot=0)

##### Exploring the Top 5 Content Types

In [ ]:
df = pd.DataFrame(counts['counts'][1]['fieldValues']).head()
df.columns=['type', 'count']
df.plot.bar(x='type', 
            y='count', 
            rot=45, 
            color=['C0', 'C1', 'C2', 'C4', 'C5'], 
            legend=False)

### Looking at a Specific User's Content

![User Exploration!](./imgs/magnifying-glass.avif)

- Users hold content, that means examining user content is very important
- As an administrator, you can stage, modify and delete user content

In [ ]:
user = gis.users.get("uc2023sharing")

In [ ]:
items = user.items(max_items=10000)
print(f"This users: {user.username} has {len(items)} items!")

#### Look at the Sharing and Types

In [ ]:
counts = content_manager.advanced_search(
    query=f"owner:{user.username} accountid:{gis.properties.id}", 
    count_fields="type, access", 
    count_size=200)

**Sharing Break Down**

In [ ]:
data = counts['counts'][0]['fieldValues']
df = pd.DataFrame(data, )
df.columns=['access', 'count']
df.plot.bar(x='access', y='count', rot=0)

**Item Type Breakdown**

In [ ]:
df = pd.DataFrame(counts['counts'][1]['fieldValues']).head(10)
df.columns=['type', 'count']
df.plot.bar(x='type', 
            y='count', 
            rot=45, 
            color=['C0', 'C1', 'C2', 'C4', 'C5'], 
            legend=False)

**Find Private Items with Relationships**

In [ ]:
query=f'owner:{user.username}  access:private accountid:{gis.properties.id}  type:("Feature Collection" OR "Feature Service")'
target_items = content_manager.advanced_search(
    query=query, 
    max_items=-1)['results']

In [ ]:
directions = ['forward', 'reverse']
relationships = ['Service2Data','Solution2Item']

In [ ]:
removable_items = []
dependant_items = {}
for item in target_items[:20]:
    related_items = []
    for direction in directions:
        for rel in relationships:
            try:
                related_items.extend(item.related_items(rel_type=rel, 
                                                        direction=direction))
            except:
                ...
    
    if len(related_items) == 0:
        removable_items.append(item) 
    else:
        if item.id in dependant_items:
            dependant_items[item.id]['rels'].extend(related_items)
        else:
            dependant_items[item.id] = {}
            dependant_items[item.id] = {'source' : item}
            dependant_items[item.id]['rels'] = related_items

In [ ]:
dependant_items[list(dependant_items.keys())[0]]

- when deleting items, you must clean up the children!

### Adding and Publishing Content

![Add and Publish Content!](./imgs/automation_zoolander.jpg)

- So far, we have looked at other people's content 
- Users with creator+ permissions can add and publish new content
- The Python API removes those clicks, and no one likes clicks

In [ ]:
import uuid
username = "a" + uuid.uuid4().hex[:5] + "z"
old_pw = "m3g4S3CUR3!2"
um = gis.users
user = um.create(username=username,
                 password=old_pw,
                 firstname="Rando",
                 lastname="Account",
                 email="python@esri.com")
user

##### Setting a User's Security Question

In [ ]:
pw = uuid.uuid4().hex[:5]+ "!aA"
user.update(security_question=1,
            security_answer=uuid.uuid4().hex)

##### Reset a User's Password

In [ ]:
user.reset(password=old_pw,
           new_password=pw)

In [ ]:
GIS(username=user.username, password=pw, set_active=False).users.me

##### Add Content to New User

- `ItemProperties` is a defined data class to assist you with adding new content
- `ItemTypeEnum` is an enumeration with common item types

In [ ]:
item = gis.content.add(
                        item_properties= ItemProperties(
                            title="Madison Urban Areas", 
                            item_type=ItemTypeEnum.SHAPEFILE),
                        data="C:\GIS\Madison_Urban_Area_Boundary.zip",
                        owner=user)
item

##### Publishing Content

In [ ]:
pitem = item.publish({"name" : "madison_uab"})
pitem

##### Adding/Publishing Tabular Content

In [ ]:
import io, uuid
import pandas as pd

In [ ]:
buffer = io.StringIO()
df = pd.read_csv("./data/banklist.csv")
df.to_csv(buffer)

In [ ]:
item_failed_banks = gis.content.add(
        item_properties={
            "type": "CSV",
            "title": "Failed Banks",
            "fileName": f"failedbanks{uuid.uuid4().hex[:5]}.csv",
        },
        data=buffer,
)

In [ ]:
item_failed_banks

In [ ]:
analyzed = gis.content.analyze(item=item_failed_banks)
analyzed.keys()

In [ ]:
publish_parameters = analyzed['publishParameters']
publish_parameters[
        'name'
    ] = f"Failed_Banks_{uuid.uuid4().hex[:2]}"  # this needs to be updated
publish_parameters['locationType'] = "none"  # this makes it a hosted table

In [ ]:
published_item = item_failed_banks.publish(publish_parameters)
published_item

##### Reassigning Items

In [ ]:
published_item.reassign_to(target_owner=user.username)

##### Updating Items

- Items once published can be modified


In [ ]:
pitem.update(thumbnail=r"./imgs/dino_icon.png")
pitem

##### Sharing is Caring

- items can be shared out to the world organization, or groups

In [ ]:
pitem.shared_with

In [ ]:
pitem.share(everyone= False,
            org=True,
            groups=None)

In [ ]:
pitem.shared_with

In [ ]:
pitem.share(everyone=True, org=True, groups=None)
pitem.shared_with

In [ ]:
grp = gis.groups.create(title='groupdemoshare', tags='tags')
pitem.share(everyone=False, org=False, groups=[grp])
pitem.shared_with

In [ ]:
grp.delete()
pitem.shared_with

In [ ]:
pitem.delete()
item.delete()

#### Reporting on Content

- as administrators, you have the ability to report on user's information

In [ ]:
import datetime
admin = gis.users.me
report_item = admin.report("content", 
                           datetime.datetime.now() - datetime.timedelta(days=10))
report_item

In [ ]:
fp = report_item.get_data()
df = pd.read_csv(fp)
q = df['View Counts'] < 1
df[q]['Item Type'].value_counts()

#### Removing Content

In [ ]:
cm:ContentManager = gis.content
cm

##### Delete on an Item Itself

In [ ]:
item = gis.users.get("uc2023sharing").items()[1]
item

In [ ]:
item_id = item.id
item_id

In [ ]:
item.delete()

##### Demo Removing All Hosted Services

- examine the staging script 
- understand the removal process

In [ ]:
import concurrent.futures

In [ ]:
services = gis.content.search("owner:uc2023sharing type:'Feature Service'", max_items=100)
fn = lambda item:item.delete()
with concurrent.futures.ThreadPoolExecutor(10) as tp:
    jobs = {}
    for service in services:
        future = tp.submit(fn, **{'item' : service})
        jobs[future] = service
        del service
    for job in concurrent.futures.as_completed(jobs):
        itemid = jobs[job].id
        print(f"Removing item: {itemid} resulted as: {job.result()}")
print('completed')

In [ ]:
[item.delete() for item in user.items()]
user.delete()